# users table에 패스워드 넣기

In [1]:
!pip install pymysql > /dev/null

In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

import json
with open('mysql.json', 'r') as fp:
    config_str = fp.read()
config = json.loads(config_str)

import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

Saving mysql.json to mysql.json


## 패스워드 암호화

- Secure Hash Algorithm(SHA-256)

In [6]:
import hashlib

pwd = '1234'
pwd_sh256 = hashlib.sha256(pwd.encode())
pwd_sh256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- Base64로 인코딩

In [7]:
import base64

base64.b64encode(pwd_sh256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [8]:
hashed_pwd = base64.b64encode(pwd_sh256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [10]:
len(hashed_pwd)

44

- 패스워드를 생성하는 함수

In [11]:
import hashlib, base64

def gen_pwd(pwd):
    pwd_sh256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sh256.digest()).decode('utf-8')
    return hashed_pwd

In [12]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## users table에 패스워드 넣기
- uid + 21

In [14]:
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('gdhong2',),
 ('jbpark',),
 ('jbpark2',),
 ('wjlee',))

In [18]:
sql_update = 'update users set pwd=%s where uid=%s'
for line in results:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql_update, (pwd, uid))
conn.commit()

- email 추가 uid@mc.com

In [19]:
sql_email = 'update users set email=%s where uid=%s'
for line in results:
    uid = line[0]
    email = f'{uid}@mc.com'
    cur.execute(sql_email, (email, uid))
conn.commit()

## 로그인

In [20]:
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'

- Case 1) uid가 없는 경우

In [21]:
# result is Null
uid = 'fool'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

- Case 2) 비밀번호가 틀린 경우

In [22]:
uid, pwd = 'djy', 'djy12'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [23]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('login')
else:
    print('incorrect')

incorrect


- Case 3) OK

In [24]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [25]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('login')
else:
    print('incorrect')

login


- 3가지 경우를 합침

In [29]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('login')
    else:
        print('incorrect')
else:
    print('wrong uid')

login


- 예외 처리를 통해 해결하는 경우

In [34]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('login')
    else:
        print('incorrect')
except:
    print('wrong uid')

login


In [35]:
cur.close()
conn.close()